In [ ]:
import yfinance as yf
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
import lightgbm as lgb

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
def calculate_indicators(data):
    """Berechnet eine Vielzahl von Indikatoren."""
    # RSI
    delta = data['Close'].diff()
    gain = delta.where(delta > 0, 0).rolling(window=14).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=14).mean()
    rs = gain / loss
    data['RSI'] = 100 - (100 / (1 + rs))

    # EMA
    data['EMA9'] = data['Close'].ewm(span=9, adjust=False).mean()
    # data['EMA14'] = data['Close'].ewm(span=14, adjust=False).mean()
    data['EMA21'] = data['Close'].ewm(span=21, adjust=False).mean()

    # Bollinger Bänder
    sma20 = data['Close'].rolling(window=20).mean()
    rolling_std = data['Close'].rolling(window=20).std()
    data['Upper Band'] = sma20 + (2 * rolling_std)
    data['Lower Band'] = sma20 - (2 * rolling_std)

    # ATR
    data['High-Low'] = data['High'] - data['Low']
    data['High-PrevClose'] = abs(data['High'] - data['Close'].shift(1))
    data['Low-PrevClose'] = abs(data['Low'] - data['Close'].shift(1))
    data['TR'] = data[['High-Low', 'High-PrevClose', 'Low-PrevClose']].max(axis=1)
    data['ATR'] = data['TR'].rolling(window=14).mean()

    # VWAP
    data['Typical_Price'] = (data['High'] + data['Low'] + data['Close']) / 3
    data['Cum_TPxV'] = (data['Typical_Price'] * data['Volume']).cumsum()
    data['Cum_Volume'] = data['Volume'].cumsum()
    data['VWAP'] = data['Cum_TPxV'] / data['Cum_Volume']

    # Stochastic Oscillator
    low_min = data['Low'].rolling(window=14).min()
    high_max = data['High'].rolling(window=14).max()
    data['Stochastic'] = 100 * ((data['Close'] - low_min) / (high_max - low_min))

    # MACD
    short_ema = data['Close'].ewm(span=12, adjust=False).mean()
    long_ema = data['Close'].ewm(span=26, adjust=False).mean()
    data['MACD'] = short_ema - long_ema
    data['Signal Line'] = data['MACD'].ewm(span=9, adjust=False).mean()

    # Lag-Werte
    for lag in range(1, 4):
        data[f'Close_lag{lag}'] = data['Close'].shift(lag)

    # Prozentuale Änderungen
    data['Pct_Change'] = data['Close'].pct_change()

    # Dropping NaN
    return data.dropna()

In [ ]:

data = calculate_indicators(data)
data_clean = data.dropna()